In [ ]:
import random
import subprocess
import time
import os

# 1. 克隆仓库 (如果仓库不存在)
if not os.path.exists("word-passgame"):
    !git clone https://github.com/peter-zx/word-passgame.git
else:
    print("仓库已存在，跳过克隆。")

# 2. 进入仓库目录
%cd word-passgame

# 3. 安装依赖
!pip install flask pyngrok

# 4. 随机选择 Flask 端口
def get_random_port():
    return random.randint(5000, 9000)

flask_port = get_random_port()

# 5. 创建 ngrok.yml 文件
ngrok_config = f"""
version: "2"
authtoken: "2ucW2Xd4dL6XACNAWeyBoDlBO9g_6g3zZJeHw61HUmfafWCiV" # 替换为你的 token
tunnels:
  my_tunnel:
    addr: {flask_port}
    proto: http
"""

with open("ngrok.yml", "w") as f:
    f.write(ngrok_config)

# 6. 启动 Flask 应用，通过环境变量传递端口号
print(f"启动 Flask 服务，端口：{flask_port}...")
os.environ["FLASK_PORT"] = str(flask_port)  # 设置环境变量
!python main.py &

# 7. 等待 Flask 启动完成 (增加等待时间)
time.sleep(10)

# 8. 检查本地服务是否可用
!curl http://localhost:$flask_port || echo "本地服务未启动"

# 9. 安装 ngrok (如果需要)
if subprocess.run(["which", "ngrok"], capture_output=True).returncode != 0:
    !pip install pyngrok

# 10. 启动 ngrok 内网穿透 (使用 ngrok.yml 配置文件)
print("启动 ngrok 内网穿透...")
!ngrok start --all

# ngrok 会在终端输出 public URL，手动复制即可。

### 下面这个不要点 这个是备用的

In [ ]:
# 1. 克隆仓库并进入目录
!git clone https://github.com/peter-zx/word-passgame.git
%cd word-passgame

# 2. 安装依赖
!pip install flask
!pip install pyngrok

# 3. 导入并运行 Flask 应用
from main import app
import threading
import time

# 在线程中启动 Flask，关闭调试模式
print("启动 Flask 服务...")
thread = threading.Thread(target=lambda: app.run(host="0.0.0.0", port=9527, debug=False), daemon=True)
thread.start()

# 等待 Flask 启动完成
time.sleep(5)

# 检查本地服务是否可用
!curl http://localhost:9527 || echo "本地服务未启动"

# 4. 设置 ngrok 内网穿透
from pyngrok import ngrok

ngrok.set_auth_token("2ucW2Xd4dL6XACNAWeyBoDlBO9g_6g3zZJeHw61HUmfafWCiV")  # 替换为你的 token
http_tunnel = ngrok.connect(9527)
public_url = http_tunnel.public_url
print("ngrok 公共 URL:", public_url)